In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import norm, qmc
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import yfinance
import datetime
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import json 

pd.set_option('display.max_columns', None)


ROOT = Path.cwd().parent.resolve()
print(ROOT)  
MODEL_DIR = ROOT / "src" / "models"
os.makedirs(MODEL_DIR, exist_ok=True)                       
print(MODEL_DIR)
PROCESSED_DATA = ROOT / "data" / "spy_option" / "processed"     # path to processed data 
OUTPUT = ROOT / "outputs"


/Users/karandeepsonewane/Projects/Option Pricing using Neural Network
/Users/karandeepsonewane/Projects/Option Pricing using Neural Network/src/models


In [2]:
pred_test = pd.read_csv(OUTPUT /"preds_test.csv")
pred_test

,Unnamed: 0,optionSymbol,expiration,dte,intrinsicValue,extrinsicValue,strike,bid,ask,mid,underlyingPrice,spot_price,snapshot_date,volume,inTheMoney,t_ann,M,C,C/K,VIX,y_ck,pred_mse,pred_wmse,error_mse,error_wmse,sq_error_mse,sq_error_wmse,pred_ck_mse,pred_ck_wmse,pred_ck_mix,err_mse,err_wmse,err_mix,abs_err_mse,abs_err_wmse,abs_err_mix,sq_error_mix
0,980328,SPY260630C00445000,1782850500,229.0,227.04,12.645,445.0,238.20,241.17,239.685,672.04,672.04,2025-11-13,0,True,0.627397,0.412244,239.685,0.538618,0.1751,0.538618,0.529897,0.549690,0.008721,0.011072,7.606093e-05,1.225861e-04,0.529897,0.549690,0.539591,-0.008721,0.011072,0.000973,0.008721,0.011072,0.000973,9.462383e-07
1,980329,SPY260630C00440000,1782850500,229.0,232.04,12.375,440.0,242.94,245.89,244.415,672.04,672.04,2025-11-13,0,True,0.627397,0.423543,244.415,0.555489,0.1751,0.555489,0.546865,0.566924,0.008624,0.011436,7.436648e-05,1.307736e-04,0.546865,0.566924,0.556793,-0.008624,0.011436,0.001304,0.008624,0.011436,0.001304,1.700806e-06
2,980330,SPY260630C00430000,1782850500,229.0,242.04,11.810,430.0,246.35,261.35,253.850,672.04,672.04,2025-11-13,0,True,0.627397,0.446533,253.850,0.590349,0.1751,0.590349,0.582154,0.602619,0.008195,0.012270,6.716068e-05,1.505479e-04,0.582154,0.602619,0.592548,-0.008195,0.012270,0.002200,0.008195,0.012270,0.002200,4.838197e-06
3,980331,SPY260630C00420000,1782850500,229.0,252.04,11.240,420.0,255.78,270.78,263.280,672.04,672.04,2025-11-13,0,True,0.627397,0.470063,263.280,0.626857,0.1751,0.626857,0.619386,0.640006,0.007472,0.013148,5.582512e-05,1.728796e-04,0.619386,0.640006,0.630192,-0.007472,0.013148,0.003335,0.007472,0.013148,0.003335,1.112421e-05
4,980332,SPY260630C00410000,1782850500,229.0,262.04,10.690,410.0,265.23,280.23,272.730,672.04,672.04,2025-11-13,0,True,0.627397,0.494161,272.730,0.665195,0.1751,0.665195,0.658685,0.679227,0.006510,0.014032,4.238268e-05,1.968888e-04,0.658685,0.679227,0.669777,-0.006510,0.014032,0.004582,0.006510,0.014032,0.004582,2.099024e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281283,1261611,SPY260320C00415000,1774037700,45.0,274.53,2.265,415.0,275.39,278.20,276.795,689.53,689.53,2026-02-03,2,True,0.123288,0.507732,276.795,0.666976,0.1744,0.666976,0.661811,0.681333,0.005165,0.014357,2.667210e-05,2.061260e-04,0.661811,0.681333,0.669505,-0.005165,0.014357,0.002529,0.005165,0.014357,0.002529,6.395365e-06
281284,1261612,SPY260320C00410000,1774037700,45.0,279.53,2.135,410.0,280.01,283.32,281.665,689.53,689.53,2026-02-03,0,True,0.123288,0.519853,281.665,0.686988,0.1744,0.686988,0.681512,0.701558,0.005476,0.014570,2.998590e-05,2.122796e-04,0.681512,0.701558,0.689419,-0.005476,0.014570,0.002431,0.005476,0.014570,0.002431,5.911960e-06
281285,1261613,SPY260320C00405000,1774037700,45.0,284.53,2.100,405.0,284.98,288.28,286.630,689.53,689.53,2026-02-03,0,True,0.123288,0.532123,286.630,0.707728,0.1744,0.707728,0.701642,0.722171,0.006087,0.014442,3.704801e-05,2.085825e-04,0.701642,0.722171,0.709769,-0.006087,0.014442,0.002041,0.006087,0.014442,0.002041,4.165852e-06
281286,1261614,SPY260320C00445000,1774037700,45.0,244.53,2.480,445.0,245.60,248.42,247.010,689.53,689.53,2026-02-03,0,True,0.123288,0.437936,247.010,0.555079,0.1744,0.555079,0.552099,0.568051,0.002980,0.012972,8.879652e-06,1.682835e-04,0.552099,0.568051,0.558431,-0.002980,0.012972,0.003353,0.002980,0.012972,0.003353,1.124141e-05


In [5]:
def mae(a, b):
    return np.mean(np.abs(a - b))

def rel_mae(y, yhat, eps=1e-8):
    return np.mean(np.abs(yhat - y) / (np.abs(y) + eps))

In [ ]:
bins = [-np.inf, -0.10, -0.02, 0.02, 0.10, np.inf]
labels = ["Deep OTM", "OTM", "ATM", "ITM", "Deep ITM"]

pred_test["moneyness_xregion"] = pd.cut(
    pred_test["M"],  # assuming M = log(S/K)
    bins=bins,
    labels=labels,
    include_lowest=True
)

pred_test["moneyness_region"].value_counts().sort_index()

moneyness_region
Deep OTM     58472
OTM          36228
ATM          29768
ITM          46939
Deep ITM    109881
Name: count, dtype: int64

In [7]:
def evaluate_by_region(df):
    rows = []
    regions = ["Deep OTM", "OTM", "ATM", "ITM", "Deep ITM"]

    for region in regions:
        mask = df["moneyness_region"] == region
        n = int(mask.sum())
        if n == 0:
            continue

        y = df.loc[mask, "y_ck"].to_numpy()

        p_mse  = df.loc[mask, "pred_ck_mse"].to_numpy()
        p_wmse = df.loc[mask, "pred_ck_wmse"].to_numpy()
        p_mix  = df.loc[mask, "pred_ck_mix"].to_numpy()

        mae_mse  = mae(y, p_mse)
        mae_wmse = mae(y, p_wmse)
        mae_mix  = mae(y, p_mix)

        rmae_mse  = rel_mae(y, p_mse)
        rmae_wmse = rel_mae(y, p_wmse)
        rmae_mix  = rel_mae(y, p_mix)

        rows.append({
            "Region": region,
            "n": n,

            "MAE_MSE": mae_mse,
            "MAE_WMSE": mae_wmse,
            "MAE_MIX": mae_mix,

            "RelMAE_MSE": rmae_mse,
            "RelMAE_WMSE": rmae_wmse,
            "RelMAE_MIX": rmae_mix,

            "ΔMAE_WMSE_vs_MSE": mae_wmse - mae_mse,
            "ΔMAE_MIX_vs_MSE":  mae_mix  - mae_mse,

            "%Imp_WMSE_vs_MSE": 100.0 * (mae_mse - mae_wmse) / (mae_mse + 1e-12),
            "%Imp_MIX_vs_MSE":  100.0 * (mae_mse - mae_mix)  / (mae_mse + 1e-12),
        })

    return pd.DataFrame(rows)

tab_all = evaluate_by_region(pred_test)
tab_all

,Region,n,MAE_MSE,MAE_WMSE,MAE_MIX,RelMAE_MSE,RelMAE_WMSE,RelMAE_MIX,ΔMAE_WMSE_vs_MSE,ΔMAE_MIX_vs_MSE,%Imp_WMSE_vs_MSE,%Imp_MIX_vs_MSE
0,Deep OTM,58472,0.001123,0.001414,0.000839,31.469982,1.071233,9.392643,0.000291,-0.000284,-25.935742,25.300114
1,OTM,36228,0.006331,0.001949,0.003640,162.664508,28.721549,91.065415,-0.004381,-0.002691,69.204911,42.502572
2,ATM,29768,0.007494,0.002207,0.004495,14.524588,4.726319,9.903514,-0.005287,-0.003000,70.545637,40.024866
3,ITM,46939,0.005688,0.001740,0.002501,0.075652,0.024567,0.037473,-0.003948,-0.003187,69.408772,56.034189
4,Deep ITM,109881,0.075518,0.087455,0.076804,3572.206410,3671.943729,3619.342421,0.011937,0.001286,-15.807228,-1.702691


In [ ]:
pred_test_liq = pred_test[pred_test["volume"] > 0]

tab_liq = evaluate_by_region(pred_test_liq)
tab_liq

,Region,n,MAE_MSE,MAE_WMSE,MAE_MIX,RelMAE_MSE,RelMAE_WMSE,RelMAE_MIX,ΔMAE_WMSE_vs_MSE,ΔMAE_MIX_vs_MSE,%Imp_WMSE_vs_MSE,%Imp_MIX_vs_MSE
0,Deep OTM,20303,0.001534,0.001747,0.001168,15.600125,0.778543,5.294624,0.000213,-0.000366,-13.905846,23.846394
1,OTM,29121,0.006651,0.001868,0.003877,137.740122,27.006902,79.853100,-0.004783,-0.002774,71.920020,41.705773
2,ATM,26727,0.007736,0.002219,0.004732,16.151725,5.255838,11.014943,-0.005516,-0.003004,71.308329,38.833576
3,ITM,20702,0.005238,0.001716,0.002491,0.079116,0.028257,0.041966,-0.003522,-0.002747,67.245738,52.446431
4,Deep ITM,16603,0.019985,0.021248,0.017210,0.025810,0.020919,0.010906,0.001263,-0.002775,-6.319423,13.886034


In [9]:
def evaluate_price_by_region(df):
    rows = []
    regions = ["Deep OTM", "OTM", "ATM", "ITM", "Deep ITM"]

    for region in regions:
        mask = df["moneyness_region"] == region
        n = int(mask.sum())
        if n == 0:
            continue

        y = df.loc[mask, "C"].to_numpy()  # or mid

        p_mse  = df.loc[mask, "pred_C_mse"].to_numpy()
        p_wmse = df.loc[mask, "pred_C_wmse"].to_numpy()
        p_mix  = df.loc[mask, "pred_C_mix"].to_numpy()

        rows.append({
            "Region": region,
            "n": n,
            "MAE_MSE":  mae(y, p_mse),
            "MAE_WMSE": mae(y, p_wmse),
            "MAE_MIX":  mae(y, p_mix),
        })

    return pd.DataFrame(rows)